In [1]:

import sys
sys.path.append('/home/techt/Desktop/a4s')


### Step 1: Initial Data Loading

In [2]:
from utils.preprocess import *
# 1. Identify the paths to the soft file and the matrix file
cohort_dir = '/media/techt/DATA/GEO/Age-Related_Macular_Degeneration/GSE67899'
soft_file, matrix_file = geo_get_relevant_filepaths(cohort_dir)

# 2. Read the matrix file to obtain background information and sample characteristics data
background_prefixes = ['!Series_title', '!Series_summary', '!Series_overall_design']
clinical_prefixes = ['!Sample_geo_accession', '!Sample_characteristics_ch1']
background_info, clinical_data = get_background_and_clinical_data(matrix_file, background_prefixes, clinical_prefixes)

# 3. Obtain the sample characteristics dictionary from the clinical dataframe
sample_characteristics_dict = get_unique_values_by_row(clinical_data)

# 4. Explicitly print out all the background information and the sample characteristics dictionary
print("Background Information:")
print(background_info)
print("Sample Characteristics Dictionary:")
print(sample_characteristics_dict)


Background Information:
!Series_title	"Delay and restoration of persistent wound-induced retinal pigmented epithelial-to-mesenchymal transition by TGF-beta pathway inhibitors: Implications for age-related macular degeneration"
!Series_summary	"This SuperSeries is composed of the SubSeries listed below."
!Series_overall_design	"Refer to individual Series"
Sample Characteristics Dictionary:
{0: ['donor id: hfRPE-020207-2', 'donor id: hfRPE-071709', 'donor id: hfRPE-081309', 'donor id: hfRPE-111109'], 1: ['plating density: 4,000 cells/cm2', 'plating density: 80,000 cells/cm2'], 2: ['passage number: 0', 'passage number: 5'], 3: ['culture time: 3 Days', 'culture time: 16 Days', 'culture time: 32 Days', 'culture time: 64 Days'], 4: ['cultureware: T75-Flask', 'cultureware: Micropourous Membrane', 'cultureware: 6-well Multiwell Plate'], 5: ['treatment: None', 'treatment: DMSO', 'treatment: 2 ng/ml FGF2', 'treatment: 500 nM A83-01', 'treatment: 500 nM A83-01 + 2ng FGF', 'treatment: 500 nM Thiaz

### Step 2: Dataset Analysis and Clinical Feature Extraction

In [3]:
is_gene_available = False
trait_row = age_row = gender_row = None  # set to different values when applicable
convert_trait = convert_age = convert_gender = None  # define the functions when applicable

# Check the dataset for gene expression data
# Based on provided information, this dataset is related to gene expressions
is_gene_available = True

# Analyzing the sample characteristics dictionary to identify row indices and conversion functions
data_dict = {
    0: ['donor id: hfRPE-020207-2', 'donor id: hfRPE-071709', 'donor id: hfRPE-081309', 'donor id: hfRPE-111109'],
    1: ['plating density: 4,000 cells/cm2', 'plating density: 80,000 cells/cm2'],
    2: ['passage number: 0', 'passage number: 5'],
    3: ['culture time: 3 Days', 'culture time: 16 Days', 'culture time: 32 Days', 'culture time: 64 Days'],
    4: ['cultureware: T75-Flask', 'cultureware: Micropourous Membrane', 'cultureware: 6-well Multiwell Plate'],
    5: ['treatment: None', 'treatment: DMSO', 'treatment: 2 ng/ml FGF2', 'treatment: 500 nM A83-01', 'treatment: 500 nM A83-01 + 2ng FGF', 'treatment: 500 nM Thiazovivin', 'treatment: 500 nM Thiazovivin + 2ng FGF', 'treatment: 200 nM LDN193189', 'treatment: 200 nM LDN193189 + 2ng FGF', 'treatment: 5 mM XAV939', 'treatment: 5 mM XAV939 + 2ng FGF']
}

# No information directly related to 'Age-Related_Macular_Degeneration', 'age', or 'gender' in the given dictionary
trait_row = age_row = gender_row = None

# Define functions to convert data types
def convert_trait(value):
    return None

def convert_age(value):
    return None

def convert_gender(value):
    return None

# Save metadata
save_cohort_info('GSE67899', './preprocessed/Age-Related_Macular_Degeneration/cohort_info.json', is_gene_available, trait_row is not None)

# Since trait_row is None, skip the clinical feature extraction part
